import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob

In [3]:
# keys and tokens from the Twitter Dev Console 
# todo - add to file so not checked into github
CONSUMER_KEY = 'oUkORoc3PPmAChcnxwfSKbRFL'
CONSUMER_SECRET = 'PmASGyF92keg7uTx8j4sNvRtWxzvw8Xm88rpxN7AqECj7YBLDe'
ACCESS_TOKEN = '65187050-L0WLjPJCAGuvcfQCYhqtJ95MtCPezJGWzhN9niDBa'
ACCESS_TOKEN_SECRET = 'C8OsEpteeL0s9xXoUU5wiip4plExftfIT72qfV7tc3xMf'

In [4]:
# Next we create a generic Twitter class for our analysis:

# First we create a TwitterClient class. 
# This class contains all the methods to interact with Twitter API and allows us to parse tweets.
# We then use an initialization (__init__) function to handle the authentication of Twitter's API client.

class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        initialization our method (which DB file didn't do in asynch ;))
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = CONSUMER_KEY
        consumer_secret = CONSUMER_SECRET
        access_token = ACCESS_TOKEN
        access_token_secret =  ACCESS_TOKEN_SECRET
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet): 
        # This function cleans tweet text by removing hyperlinks and special characters
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        This function classifies sentiment of passed tweet 
        using textblob's predefined and built sentiment method
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity < 0:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        ''' 
        This function then fetches tweets and parses them, using lists and dictionaries to store them 
        '''
        # empty list to store parsed tweets 
        tweets = []
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {}
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))

# Then we define and call our main function to get back all our tweets:
def main(q): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets mentioning a specific phrase
    tweets = api.get_tweets(query = q, count = 5000)
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:5]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:5]: 
        print(tweet['text']) 
if __name__ == "__main__": 
    # calling main function 
    main('sad') # we parse our query into our main function. Here a simple test to display our program works correctly

Error: Authentication Failed


NameError: name 'tweepy' is not defined

As we would expect, we see much more negative sentiment tweets when we query 1,000 tweets from twitter under the search term "sad". We can verify this again by looking for "happy" tweets:

In [ ]:
main('happy')

Happy is overwhelmingly positive! But now, let's take a look at 5,000 tweets (so we'll go back, adjust our function, and run it again, based on the search term of "Iran Nuclear Deal". Why? We want to see how the twitterverse feels about this right now, as it will be a proxy for whether people are thinking or speaking about Iran positively or negatively:

In [ ]:
main('iran nuclear deal')

Interestingly, we can see that Iran is typically referred to positively. This can be deceiving however, as people could be positively speaking about withdrawl from the iran nuclear deal, and may have negative sentiment toward Iran, or toward the deal as a whole. This is one limitation of sentiment analysis, which we can see in action.

Let's try this again. Lately, LaMelo Ball - the younger brother of Lonzo Ball, a professional NBA player and a top amateur basketball player himself - has seen his school's opponents canceling games against him and his team because he declared professional previously. That is, many schools don't believe he is a "valid" amateur athlete. Thus, let's see how the twitterverse feels about LaMelo Ball here:

In [ ]:
main('LaMelo Ball')

Again, we see some limitations of our textblob package. At least one tweet in the positive sentiment analysis looks negative, but both our tweets in negative tweets appear neutral to positive? One thing we can do is tweak our sentiment score filters. The scores are on a scale of -1 to +1, so let's adjust them in our function above to 0.5 (were previously 0) and -0.5, to get really positive and really negative scores only:

In [ ]:
class TwitterClient(object): 
    def __init__(self): 
        consumer_key = CONSUMER_KEY
        consumer_secret = CONSUMER_SECRET
        access_token = ACCESS_TOKEN
        access_token_secret =  ACCESS_TOKEN_SECRET
        
        try: 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet): 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        if analysis.sentiment.polarity > 0.5: # here we changed our positive score threshold
            return 'positive'
        elif analysis.sentiment.polarity < -0.5: # here we changed our negative score threshold
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        tweets = []
  
        try: 
            fetched_tweets = self.api.search(q = query, count = count)
  
            for tweet in fetched_tweets: 
                parsed_tweet = {}
  
                parsed_tweet['text'] = tweet.text 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                if tweet.retweet_count > 0: 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            return tweets 
  
        except tweepy.TweepError as e: 
            print("Error : " + str(e))

def main(q): 
    api = TwitterClient() 
    tweets = api.get_tweets(query = q, count = 5000)
  
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
  
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:5]: 
        print(tweet['text']) 
  
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:5]: 
        print(tweet['text']) 
        
if __name__ == "__main__": 
    main('LaMelo Ball')

No negative tweets about LaMelo? Maybe people really like the kid :).
    
Let's try with a more polarizing, and prominent figure: Donald Trump. How does the twitterverse feel about The Donald today?

In [ ]:
main('Donald Trump')

Here we see something interesting. With our main threshold set to 0.5 and -0.5 for the scores of sentiment, we see that Donal Trump is being tweeted about in a marginally negative context, when analysing the latest 5,000 tweets about him. If, however, we were to return our score thresholds to zero - what do people tend to think about Trump?

In [ ]:
class TwitterClient(object): 
    def __init__(self): 
        consumer_key = CONSUMER_KEY
        consumer_secret = CONSUMER_SECRET
        access_token = ACCESS_TOKEN
        access_token_secret =  ACCESS_TOKEN_SECRET
        
        try: 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet): 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        if analysis.sentiment.polarity > 0: # here we changed our positive score threshold
            return 'positive'
        elif analysis.sentiment.polarity < 0: # here we changed our negative score threshold
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        tweets = []
  
        try: 
            fetched_tweets = self.api.search(q = query, count = count)
  
            for tweet in fetched_tweets: 
                parsed_tweet = {}
  
                parsed_tweet['text'] = tweet.text 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                if tweet.retweet_count > 0: 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            return tweets 
  
        except tweepy.TweepError as e: 
            print("Error : " + str(e))

def main(q): 
    api = TwitterClient() 
    tweets = api.get_tweets(query = q, count = 5000)
  
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
  
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:5]: 
        print(tweet['text']) 
  
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:5]: 
        print(tweet['text']) 
        
if __name__ == "__main__": 
    main('Donald Trump')

It appears that the latest tweets about the POTUS are mostly positive. Finally, we look at one last objective measure: the stock market.

In [ ]:
main('stock market')

Good news today, traders, the sentiment on The Street is upbeat!

# Conclusion
What can we learn from all of this? First and foremost, we can learn that SA (sentiment analysis) is tough. We as humans read things and interpret them based on our points of view. A computational method of analysis for that is inherently going to contain some biases, and so the question becomes - how can we overcome those biases to be more objective? Things like language, context, past history, sarcasm, etc. are all phenomena we consider as humans when we assess the "sentiment" of a statement. When a computer does that, it can lead to some interesting results like with LaMelo Ball, the Iran Deal, or even Donald Trump.

We also see, though, that in the case of Donald Trump for instance - the textblob SA package does a decent job. Of the latest 5,000 tweets, only about 30% of them are positive when referencing Trump, to 20% negative. That means about half are "neutral". We could potentially dive deeper and look at the netural tweets to see if they are coming from news organizations or supposedly "unbiased" journalists, to disprove "fake news" and bias claims for instance. However, we can see here that this is a narrow margin, which holds with our view of Trump in the publich sphere. He can sometimes be positive, but narrowly, thanks to his base. That holds true here (sic).

So in closing, we took in live tweets, bucketed them by their sentiment score, and then played around with different queries to test our program. Business, politicians, and even public service officers can do the same to aid them in the completion of their daily jobs!